<a href="https://colab.research.google.com/github/TNTanKhai/TTNT/blob/main/Tuan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import các thư viện cần thiết
import copy    # Để tạo bản sao sâu của danh sách
import math    # Để sử dụng giá trị vô cực trong thuật toán
import random  # Để tạo số ngẫu nhiên (nếu cần)
import numpy   # Để hiển thị bảng cờ dạng ma trận đẹp hơn

# Định nghĩa các ký hiệu cho trò chơi
X = "X"        # Người chơi X
O = "O"        # Người chơi O
EMPTY = None   # Ô trống trên bàn cờ
user = None    # Biến lưu ký hiệu của người dùng (X hoặc O)
ai = None      # Biến lưu ký hiệu của máy (đối thủ của người dùng)
board_size = 3 # Kích thước bàn cờ (mặc định 3x3)

def initial_state():
    """
    Tạo trạng thái ban đầu của bàn cờ.
    Tất cả các ô đều trống.
    """
    # Tạo ma trận board_size x board_size với tất cả giá trị EMPTY
    return [[EMPTY for _ in range(board_size)] for _ in range(board_size)]

def player(board):
    """
    Xác định người chơi có lượt đi tiếp theo.
    """
    count = 0  # Biến đếm số nước đi đã thực hiện

    # Duyệt qua tất cả các ô trên bàn cờ
    for i in board:      # i là một hàng trong bàn cờ
        for j in i:      # j là một ô trong hàng
            if j:        # Nếu ô không trống (có giá trị X hoặc O)
                count += 1  # Tăng biến đếm

    # Logic: X luôn đi trước, nếu số nước đi lẻ thì O đi, chẵn thì X đi
    if count % 2 != 0:   # Nếu số nước đi là số lẻ
        return ai        # Trả về AI (vì đã đến lượt của đối thủ)
    return user          # Ngược lại, trả về User

def actions(board):
    """
    Trả về tập hợp tất cả các nước đi có thể thực hiện.
    """
    res = set()  # Tạo một tập hợp rỗng để lưu các nước đi hợp lệ

    # Duyệt qua tất cả các ô trên bàn cờ
    for i in range(board_size):      # Duyệt theo hàng
        for j in range(board_size):  # Duyệt theo cột
            if board[i][j] == EMPTY: # Nếu ô hiện tại trống
                res.add((i, j))      # Thêm vị trí (i,j) vào tập hợp kết quả

    return res  # Trả về tập hợp các nước đi có thể

def result(board, action):
    """
    Tạo bàn cờ mới sau khi thực hiện nước đi.
    """
    # Xác định người chơi hiện tại
    curr_player = player(board)

    # Tạo bản sao sâu của bàn cờ hiện tại để không làm thay đổi bàn cờ gốc
    result_board = copy.deepcopy(board)

    # Lấy tọa độ từ action
    (i, j) = action

    # Đặt ký hiệu của người chơi hiện tại vào vị trí (i,j)
    result_board[i][j] = curr_player

    return result_board  # Trả về bàn cờ mới

def get_horizontal_winner(board):
    """
    Kiểm tra người thắng theo hàng ngang.
    """
    winner_val = None  # Biến lưu người thắng, ban đầu là None

    # Duyệt qua từng hàng
    for i in range(board_size):
        # Giả sử người thắng là người ở ô đầu tiên của hàng
        winner_val = board[i][0]

        # Kiểm tra tất cả các ô trong hàng có giống ô đầu tiên không
        for j in range(board_size):
            if board[i][j] != winner_val:  # Nếu có ô khác giá trị
                winner_val = None           # Đặt lại thành None
                break                       # Thoát vòng lặp sớm

        # Nếu tìm thấy người thắng, trả về kết quả ngay
        if winner_val:
            return winner_val

    return winner_val  # Trả về None nếu không có người thắng theo hàng

def get_vertical_winner(board):
    """
    Kiểm tra người thắng theo hàng dọc.
    """
    winner_val = None  # Biến lưu người thắng, ban đầu là None

    # Duyệt qua từng cột
    for i in range(board_size):
        # Giả sử người thắng là người ở ô đầu tiên của cột
        winner_val = board[0][i]

        # Kiểm tra tất cả các ô trong cột có giống ô đầu tiên không
        for j in range(board_size):
            if board[j][i] != winner_val:  # Nếu có ô khác giá trị
                winner_val = None           # Đặt lại thành None
                break                       # Thoát vòng lặp sớm

        # Nếu tìm thấy người thắng, trả về kết quả ngay
        if winner_val:
            return winner_val

    return winner_val  # Trả về None nếu không có người thắng theo cột

def get_diagonal_winner(board):
    """
    Kiểm tra người thắng theo đường chéo.
    """
    winner_val = None  # Biến lưu người thắng

    # Kiểm tra đường chéo chính (từ trái trên sang phải dưới)
    winner_val = board[0][0]  # Ô đầu tiên của đường chéo
    for i in range(board_size):
        if board[i][i] != winner_val:  # Nếu có ô khác giá trị
            winner_val = None           # Đặt lại thành None
            break                       # Thoát vòng lặp

    # Nếu tìm thấy người thắng, trả về kết quả
    if winner_val:
        return winner_val

    # Kiểm tra đường chéo phụ (từ phải trên sang trái dưới)
    winner_val = board[0][board_size - 1]  # Ô đầu tiên của đường chéo phụ
    for i in range(board_size):
        j = board_size - 1 - i  # Tính vị trí cột cho đường chéo phụ
        if board[i][j] != winner_val:  # Nếu có ô khác giá trị
            winner_val = None           # Đặt lại thành None
            break                       # Thoát vòng lặp

    return winner_val  # Trả về kết quả (có thể là None)

def winner(board):
    """
    Xác định người thắng cuộc của trò chơi.
    """
    # Kiểm tra lần lượt: hàng ngang, hàng dọc, đường chéo
    # Nếu không có ai thắng thì trả về None
    winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board) or None
    return winner_val

def terminal(board):
    """
    Kiểm tra trò chơi đã kết thúc chưa.
    """
    # Nếu đã có người thắng, trò chơi kết thúc
    if winner(board) != None:
        return True

    # Kiểm tra nếu còn ô trống nào không
    for i in board:      # Duyệt qua từng hàng
        for j in i:      # Duyệt qua từng ô trong hàng
            if j == EMPTY:  # Nếu tìm thấy ô trống
                return False  # Trò chơi chưa kết thúc

    # Nếu không còn ô trống nào, trò chơi kết thúc (hòa)
    return True

def utility(board):
    """
    Tính điểm utility của bàn cờ.
    """
    winner_val = winner(board)  # Xác định người thắng

    if winner_val == X:  # Nếu X thắng
        return 1         # Trả về 1
    elif winner_val == O:  # Nếu O thắng
        return -1        # Trả về -1
    return 0             # Nếu hòa, trả về 0

def maxValue(state):
    """
    Hàm MAX trong thuật toán Minimax - tìm nước đi tốt nhất cho MAX (X).
    """
    # Nếu đây là trạng thái kết thúc, trả về utility
    if terminal(state):
        return utility(state)

    v = -math.inf  # Khởi tạo giá trị tốt nhất là âm vô cực

    # Duyệt qua tất cả các nước đi có thể
    for action in actions(state):
        # Gọi đệ quy hàm MIN và cập nhật giá trị tốt nhất
        v = max(v, minValue(result(state, action)))

    return v  # Trả về giá trị tốt nhất tìm được

def minValue(state):
    """
    Hàm MIN trong thuật toán Minimax - tìm nước đi tốt nhất cho MIN (O).
    """
    # Nếu đây là trạng thái kết thúc, trả về utility
    if terminal(state):
        return utility(state)

    v = math.inf  # Khởi tạo giá trị tốt nhất là dương vô cực

    # Duyệt qua tất cả các nước đi có thể
    for action in actions(state):
        # Gọi đệ quy hàm MAX và cập nhật giá trị tốt nhất
        v = min(v, maxValue(result(state, action)))

    return v  # Trả về giá trị tốt nhất tìm được

def minimax(board):
    """
    Thuật toán Minimax - tìm nước đi tối ưu cho người chơi hiện tại.
    """
    current_player = player(board)  # Xác định người chơi hiện tại

    if current_player == X:  # Nếu là lượt của X (MAX player)
        min_val = -math.inf  # Khởi tạo giá trị tốt nhất là âm vô cực

        # Duyệt qua tất cả các nước đi có thể
        for action in actions(board):
            # Tính giá trị của nước đi này bằng cách gọi hàm MIN
            check = minValue(result(board, action))

            # Nếu nước đi này tốt hơn, cập nhật
            if check > min_val:
                min_val = check    # Cập nhật giá trị tốt nhất
                move = action      # Lưu nước đi tốt nhất

    else:  # Nếu là lượt của O (MIN player)
        max_val = math.inf  # Khởi tạo giá trị tốt nhất là dương vô cực

        # Duyệt qua tất cả các nước đi có thể
        for action in actions(board):
            # Tính giá trị của nước đi này bằng cách gọi hàm MAX
            check = maxValue(result(board, action))

            # Nếu nước đi này tốt hơn, cập nhật
            if check < max_val:
                max_val = check    # Cập nhật giá trị tốt nhất
                move = action      # Lưu nước đi tốt nhất

    return move  # Trả về nước đi tối ưu

if __name__ == "__main__":
    """
    Hàm main - khởi chạy trò chơi.
    """
    # Cho phép người dùng chọn kích thước bàn cờ
    while True:
        try:
            size_input = input("Enter board size (3 for 3x3, 4 for 4x4, etc.): ")
            board_size = int(size_input)  # Chuyển đổi input thành số
            if board_size >= 3:  # Kiểm tra kích thước hợp lệ
                break
            else:
                print("Size must be at least 3.")
        except ValueError:  # Bắt lỗi nếu input không phải số
            print("Please enter a valid number.")

    # Khởi tạo bàn cờ với kích thước đã chọn
    board = initial_state()
    ai_turn = False  # Biến xác định có phải lượt của AI không

    # Cho người dùng chọn ký hiệu (X hoặc O)
    print("Choose a player")
    user = input()  # Nhập ký hiệu từ người dùng

    # Xác định ký hiệu của AI dựa trên lựa chọn của người dùng
    if user == "X":
        ai = "O"  # Nếu user chọn X thì AI là O
    else:
        ai = "X"  # Nếu user chọn O thì AI là X

    # Vòng lặp chính của trò chơi
    while True:
        game_over = terminal(board)  # Kiểm tra trò chơi đã kết thúc chưa
        playr = player(board)        # Xác định người chơi hiện tại

        if game_over:  # Nếu trò chơi đã kết thúc
            winner_val = winner(board)  # Xác định người thắng

            if winner_val is None:  # Nếu không có người thắng (hòa)
                print("Game Over: Tie.")
            else:  # Nếu có người thắng
                print(f"Game Over: {winner_val} wins.")
            break  # Thoát khỏi vòng lặp, kết thúc trò chơi

        else:  # Nếu trò chơi chưa kết thúc
            if user != playr and not game_over:  # Nếu đến lượt AI
                if ai_turn:  # Nếu đã được đánh dấu là lượt của AI
                    move = minimax(board)        # AI tính nước đi tối ưu
                    board = result(board, move)  # Thực hiện nước đi
                    ai_turn = False              # Đánh dấu đã đi xong
                    print(numpy.array(board))    # Hiển thị bàn cờ

            elif user == playr and not game_over:  # Nếu đến lượt người dùng
                ai_turn = True  # Đánh dấu lượt tiếp theo là của AI
                print("Enter the position to move (row,col)")

                # Nhập tọa độ từ người dùng
                i = int(input("Row:"))  # Nhập số hàng
                j = int(input("Col:"))  # Nhập số cột

                # Kiểm tra nước đi hợp lệ và thực hiện
                if board[i][j] == EMPTY:  # Nếu ô trống
                    board = result(board, (i, j))  # Thực hiện nước đi
                    print(numpy.array(board))      # Hiển thị bàn cờ

Choose a player
Enter the position to move (row,col)
[[None None None]
 [None None 'O']
 [None None None]]
[[None None None]
 [None 'X' 'O']
 [None None None]]
Enter the position to move (row,col)


In [5]:
import os
import math

def GetWinner(board, size=3, win_length=3):
    """
    Kiểm tra và trả về người chiến thắng trên bảng.
    Tham số:
    - board: danh sách đại diện cho bảng
    - size: kích thước bảng (size x size)
    - win_length: số ô liên tiếp cần để thắng
    Trả về: "X", "O" nếu có người thắng, None nếu chưa có
    """
    # Convert board to 2D for easier checking - Chuyển board 1D thành 2D để dễ kiểm tra
    board_size = len(board)

    # Check hàng ngang - Kiểm tra các hàng
    for row in range(size):
        for col in range(size - win_length + 1):
            start_idx = row * size + col
            # Kiểm tra win_length ô liên tiếp trong hàng
            if all(board[start_idx] == board[start_idx + i] and
                   board[start_idx] in ["X", "O"]
                   for i in range(win_length)):
                return board[start_idx]  # Trả về người thắng

    # Check hàng dọc - Kiểm tra các cột
    for col in range(size):
        for row in range(size - win_length + 1):
            start_idx = row * size + col
            # Kiểm tra win_length ô liên tiếp trong cột
            if all(board[start_idx] == board[start_idx + i*size] and
                   board[start_idx] in ["X", "O"]
                   for i in range(win_length)):
                return board[start_idx]

    # Check đường chéo chính (\) - Kiểm tra đường chéo từ trái trên sang phải dưới
    for row in range(size - win_length + 1):
        for col in range(size - win_length + 1):
            start_idx = row * size + col
            # Kiểm tra win_length ô liên tiếp trên đường chéo chính
            if all(board[start_idx] == board[start_idx + i*(size+1)] and
                   board[start_idx] in ["X", "O"]
                   for i in range(win_length)):
                return board[start_idx]

    # Check đường chéo phụ (/) - Kiểm tra đường chéo từ phải trên sang trái dưới
    # SỬA LỖI: Đã sửa công thức tính chỉ số cho đường chéo phụ
    for row in range(size - win_length + 1):
        for col in range(win_length - 1, size):
            start_idx = row * size + col
            # Kiểm tra win_length ô liên tiếp trên đường chéo phụ
            # Công thức đúng: start_idx + i*(size-1)
            if all(0 <= start_idx + i*(size-1) < board_size and  # Kiểm tra chỉ số hợp lệ
                   board[start_idx] == board[start_idx + i*(size-1)] and
                   board[start_idx] in ["X", "O"]
                   for i in range(win_length)):
                return board[start_idx]

    return None  # Không có ai thắng

def PrintBoard(board, size=3):
    """
    Xóa màn hình console và in bảng hiện tại.
    """
    os.system('cls' if os.name == 'nt' else 'clear')  # Xóa màn hình: cls cho Windows, clear cho Linux/Mac

    print(f"\nTic-Tac-Toe {size}x{size}")  # In tiêu đề với kích thước bảng
    print("-" * (size * 4 - 1))  # In đường kẻ ngang phân cách

    # In từng hàng của bảng
    for i in range(0, size*size, size):
        row = board[i:i+size]  # Lấy một hàng từ board

        # Định dạng mỗi ô để canh giữa với độ rộng 3 ký tự
        formatted_row = [f"{cell:^3}" for cell in row]

        # In hàng với dấu | phân cách các ô
        print(" | ".join(formatted_row))

        # In đường kẻ ngang giữa các hàng (trừ hàng cuối)
        if i < size*size - size:
            print("-" * (size * 4 - 1))
    print()  # In dòng trống cuối cùng

def GetAvailableCells(board):
    """
    Trả về danh sách các chỉ số của các ô còn trống trên bảng.
    """
    available = []  # Danh sách chứa các ô trống

    # Duyệt qua tất cả các ô trên bảng
    for i in range(len(board)):
        # Nếu ô không phải "X" hoặc "O" thì còn trống
        if str(board[i]) not in ["X", "O"]:
            available.append(i)  # Thêm chỉ số ô trống vào danh sách

    return available

def minimax(position, depth, alpha, beta, isMaximizing, size=3, win_length=3):
    """
    Thuật toán Minimax với Alpha-Beta Pruning để tìm nước đi tối ưu.

    Tham số:
    - position: trạng thái bảng hiện tại
    - depth: độ sâu hiện tại trong cây tìm kiếm
    - alpha: giá trị tốt nhất cho người chơi tối đa hóa
    - beta: giá trị tốt nhất cho người chơi tối thiểu hóa
    - isMaximizing: True nếu lượt của người chơi tối đa hóa (X), False nếu của người tối thiểu hóa (O)
    - size: kích thước bảng
    - win_length: số ô cần để thắng

    Trả về: giá trị heuristic của vị trí hiện tại
    """
    # Kiểm tra nếu đã có người thắng
    winner = GetWinner(position, size, win_length)
    if winner is not None:
        # Trả về điểm số: cao cho X thắng, thấp cho O thắng, trừ đi độ sâu để ưu tiên thắng nhanh/thua chậm
        return 1000 - depth if winner == "X" else -1000 + depth

    # Kiểm tra nếu bảng đầy (hòa)
    available = GetAvailableCells(position)
    if len(available) == 0:
        return 0  # Hòa

    # Nếu là lượt của người chơi tối đa hóa (X)
    if isMaximizing:
        maxEval = -math.inf  # Khởi tạo giá trị tối đa là âm vô cùng

        # Duyệt qua tất cả các nước đi có thể
        for cell in available:
            original_value = position[cell]  # Lưu giá trị gốc của ô
            position[cell] = "X"  # Thử đặt X vào ô này

            # Gọi đệ quy minimax cho trạng thái mới với lượt của đối thủ
            eval_score = minimax(position, depth + 1, alpha, beta, False, size, win_length)

            # Cập nhật giá trị tối đa
            maxEval = max(maxEval, eval_score)

            # Cập nhật giá trị alpha (cắt tỉa alpha-beta)
            alpha = max(alpha, eval_score)

            position[cell] = original_value  # Khôi phục trạng thái bảng

            # Nếu beta <= alpha, cắt tỉa nhánh này
            if beta <= alpha:
                break  # Alpha-beta pruning

        return maxEval  # Trả về giá trị tốt nhất cho X

    else:  # Nếu là lượt của người chơi tối thiểu hóa (O)
        minEval = math.inf  # Khởi tạo giá trị tối thiểu là dương vô cùng

        # Duyệt qua tất cả các nước đi có thể
        for cell in available:
            original_value = position[cell]  # Lưu giá trị gốc của ô
            position[cell] = "O"  # Thử đặt O vào ô này

            # Gọi đệ quy minimax cho trạng thái mới với lượt của đối thủ
            eval_score = minimax(position, depth + 1, alpha, beta, True, size, win_length)

            # Cập nhật giá trị tối thiểu
            minEval = min(minEval, eval_score)

            # Cập nhật giá trị beta (cắt tỉa alpha-beta)
            beta = min(beta, eval_score)

            position[cell] = original_value  # Khôi phục trạng thái bảng

            # Nếu beta <= alpha, cắt tỉa nhánh này
            if beta <= alpha:
                break  # Alpha-beta pruning

        return minEval  # Trả về giá trị tốt nhất cho O

def FindBestMove(currentPosition, AI, size=3, win_length=3):
    """
    Tìm nước đi tốt nhất cho AI.

    Tham số:
    - currentPosition: trạng thái bảng hiện tại
    - AI: ký tự đại diện cho AI ("X" hoặc "O")
    - size: kích thước bảng
    - win_length: số ô cần để thắng

    Trả về: chỉ số của ô tốt nhất để đi
    """
    board_size = size * size  # Tổng số ô trên bảng

    # Nếu là nước đầu tiên, ưu tiên chọn ô trung tâm
    if len(GetAvailableCells(currentPosition)) == board_size:
        center = board_size // 2  # Chỉ số ô trung tâm
        # Kiểm tra nếu ô trung tâm còn trống
        if str(currentPosition[center]) not in ["X", "O"]:
            return center  # Trả về ô trung tâm

    # Nếu AI là X (người chơi tối đa hóa)
    if AI == "X":
        bestVal = -math.inf  # Khởi tạo giá trị tốt nhất là âm vô cùng
        bestMove = -1  # Khởi tạo nước đi tốt nhất là -1 (chưa tìm thấy)

        # Duyệt qua tất cả các ô trống
        for cell in GetAvailableCells(currentPosition):
            original_value = currentPosition[cell]  # Lưu giá trị gốc
            currentPosition[cell] = AI  # Thử đặt AI vào ô này

            # Tính giá trị minimax cho nước đi này
            moveVal = minimax(currentPosition, 0, -math.inf, math.inf, False, size, win_length)

            currentPosition[cell] = original_value  # Khôi phục trạng thái

            # Nếu giá trị này tốt hơn giá trị tốt nhất hiện tại
            if moveVal > bestVal:
                bestMove = cell  # Cập nhật nước đi tốt nhất
                bestVal = moveVal  # Cập nhật giá trị tốt nhất

    else:  # Nếu AI là O (người chơi tối thiểu hóa)
        bestVal = math.inf  # Khởi tạo giá trị tốt nhất là dương vô cùng
        bestMove = -1  # Khởi tạo nước đi tốt nhất là -1

        # Duyệt qua tất cả các ô trống
        for cell in GetAvailableCells(currentPosition):
            original_value = currentPosition[cell]  # Lưu giá trị gốc
            currentPosition[cell] = AI  # Thử đặt AI vào ô này

            # Tính giá trị minimax cho nước đi này
            moveVal = minimax(currentPosition, 0, -math.inf, math.inf, True, size, win_length)

            currentPosition[cell] = original_value  # Khôi phục trạng thái

            # Nếu giá trị này tốt hơn (nhỏ hơn) giá trị tốt nhất hiện tại
            if moveVal < bestVal:
                bestMove = cell  # Cập nhật nước đi tốt nhất
                bestVal = moveVal  # Cập nhật giá trị tốt nhất

    return bestMove  # Trả về nước đi tốt nhất

def main():
    """
    Hàm chính điều khiển toàn bộ trò chơi.
    """
    # Phần 1: Nhập kích thước bảng
    while True:
        try:
            # Yêu cầu người chơi nhập kích thước bảng
            size_input = input("Enter board size (default 3): ").strip()

            # Xử lý đầu vào: nếu để trống thì dùng mặc định 3
            if size_input == "":
                size = 3
            else:
                size = int(size_input)  # Chuyển đổi thành số nguyên

            # Kiểm tra kích thước hợp lệ
            if size >= 2:
                break  # Thoát vòng lặp nếu hợp lệ
            else:
                print("Size must be at least 2.")  # Thông báo lỗi
        except ValueError:
            print("Please enter a valid number.")  # Bắt lỗi nhập không phải số

    # Phần 2: Nhập số ô cần để thắng
    while True:
        try:
            # Yêu cầu nhập số ô cần để thắng
            win_length_input = input(f"Enter winning length (default {min(5, size)}): ").strip()

            # Xử lý đầu vào
            if win_length_input == "":
                win_length = min(5, size)  # Mặc định là 5 hoặc nhỏ hơn kích thước bảng
            else:
                win_length = int(win_length_input)

            # Kiểm tra điều kiện hợp lệ
            if 2 <= win_length <= size:
                break  # Thoát vòng lặp nếu hợp lệ
            else:
                print(f"Winning length must be between 2 and {size}.")  # Thông báo lỗi
        except ValueError:
            print("Please enter a valid number.")  # Bắt lỗi nhập không phải số

    board_size = size * size  # Tính tổng số ô trên bảng

    # Phần 3: Chọn vai trò người chơi
    while True:
        # Yêu cầu người chơi chọn X hoặc O
        player = input("Play as X or O? ").strip().upper()

        # Kiểm tra lựa chọn hợp lệ
        if player in ["X", "O"]:
            break  # Thoát vòng lặp nếu hợp lệ
        print("Invalid choice. Please enter X or O.")  # Thông báo lỗi

    # Xác định AI là đối thủ của người chơi
    AI = "O" if player == "X" else "X"

    # Phần 4: Khởi tạo bảng trò chơi
    # Tạo bảng với các số từ 1 đến board_size (dưới dạng string)
    currentGame = [str(i+1) for i in range(board_size)]

    currentTurn = "X"  # X luôn đi trước trong Tic-Tac-Toe
    counter = 0  # Đếm số nước đi đã thực hiện

    # Phần 5: Vòng lặp chính của trò chơi
    while True:
        # Nếu đến lượt AI
        if currentTurn == AI:
            # Hiển thị bảng
            PrintBoard(currentGame, size)
            print(f"AI ({AI}) is thinking...")  # Thông báo AI đang suy nghĩ

            # Tìm nước đi tốt nhất cho AI
            cell = FindBestMove(currentGame, AI, size, win_length)

            # Thực hiện nước đi của AI
            currentGame[cell] = AI

            # Chuyển lượt cho người chơi
            currentTurn = player

            # Tăng biến đếm nước đi
            counter += 1

        # Nếu đến lượt người chơi
        elif currentTurn == player:
            # Hiển thị bảng
            PrintBoard(currentGame, size)
            print(f"Your turn ({player})")  # Thông báo lượt của người chơi

            # Vòng lặp nhập nước đi của người chơi
            while True:
                try:
                    # Yêu cầu nhập vị trí
                    humanInput = input(f"Enter position (1-{board_size}): ").strip()

                    # Kiểm tra đầu vào không rỗng
                    if not humanInput:
                        print("Please enter a position.")
                        continue

                    # Chuyển đổi thành số nguyên
                    humanInput = int(humanInput)

                    # Kiểm tra vị trí hợp lệ
                    if 1 <= humanInput <= board_size:
                        # Kiểm tra ô còn trống
                        if currentGame[humanInput - 1] not in ["X", "O"]:
                            # Thực hiện nước đi của người chơi
                            currentGame[humanInput - 1] = player

                            # Chuyển lượt cho AI
                            currentTurn = AI

                            # Tăng biến đếm nước đi
                            counter += 1
                            break  # Thoát vòng lặp nhập
                        else:
                            print("Cell is already taken. Choose another.")  # Ô đã được đánh
                    else:
                        print(f"Please enter a number between 1 and {board_size}.")  # Vị trí ngoài phạm vi
                except ValueError:
                    print("Please enter a valid number.")  # Nhập không phải số

        # Phần 6: Kiểm tra kết thúc trò chơi
        # Kiểm tra có người thắng chưa
        winner = GetWinner(currentGame, size, win_length)
        if winner is not None:
            # Hiển thị bảng cuối cùng
            PrintBoard(currentGame, size)

            # Thông báo người thắng
            print(f"{winner} WON!!!")
            print(f"Total moves: {counter}")  # In tổng số nước đi
            break  # Kết thúc trò chơi

        # Kiểm tra hòa (hết ô trống)
        if len(GetAvailableCells(currentGame)) == 0:
            # Hiển thị bảng cuối cùng
            PrintBoard(currentGame, size)

            # Thông báo hòa
            print("Tie.")
            print(f"Total moves: {counter}")  # In tổng số nước đi
            break  # Kết thúc trò chơi

if __name__ == "__main__":
    """
    Điểm bắt đầu của chương trình.
    Chỉ chạy hàm main() khi file được thực thi trực tiếp.
    """
    main()

KeyboardInterrupt: Interrupted by user